In [1]:
import sys
sys.path.append("/p300s/wangmx_group/xutingfeng/SIS/")




In [14]:
import sis.model.transformer as trans 
from torch import nn
from sis.model.embedding import OnehotLayer 
import torch.nn.functional as F
import copy 
import torch 

c = copy.deepcopy

class SisModle(nn.Module):
    def __init__(self,N, d_model, d_ff, dropout, seq_length):
        super(SisModle, self).__init__()

        self.N = N 
        self.d_model = d_model
        self.d_ff = d_ff 
        self.dropout = dropout
        self.seq_length = seq_length
        # embedding layer 
        self.onehot = OnehotLayer(d_model)
        # basic layer 
        attn = trans.MultiHeadAttention(1, d_model)  # head as 1 
        ff = trans.PositionwiseFeedForward(d_model, d_ff,dropout)
        encoderlayer = trans.EncoderLayer(d_model, c(attn), c(ff), dropout)
        # transformer encoder build 
        self.encoder_SLF = trans.Encoder(c(encoderlayer), N)
        self.encoder_SRnase = trans.Encoder(c(encoderlayer), N)
        # fc layer 
        self.fc1 = nn.Linear(d_model, 1)
        self.fc2 = nn.Linear(seq_length, 1)
        
    def forward(self, x):
        x_SLF = self.onehot(x["SLF_Seq_token"])
        x_SRnase = self.onehot(x["SRnase_Seq_token"])

        x_SLF = self.fc1(self.encoder_SLF(x_SLF, None))
        x_SRnase = self.fc1(self.encoder_SRnase(x_SRnase, None))
        
        o = F.relu(torch.concat([x_SLF, x_SRnase], dim=1).squeeze(-1))
        
        last_mask = torch.concat([x["SLF_Seq_mask"], x["SRnase_Seq_mask"]], dim=1)
        o = F.sigmoid(self.fc2(o.masked_fill(last_mask, 1e-9)))
        
        return o 

In [ ]:
from sis.dataset import SISDataset
import torch 
import transformers 

sisdataset = SISDataset(root_dir="/p300s/wangmx_group/xutingfeng/SIS/sis/dataset/total_data.csv")

aa_vocab = sisdataset.aa_vocab
sis_datasetDict = sisdataset.dataset_dict

train_dataloader = torch.utils.data.DataLoader(sis_datasetDict["train"], batch_size=4)
test_dataloader = torch.utils.data.DataLoader(sis_datasetDict["test"], batch_size=4)

In [15]:
N = 4 
d_model = len(aa_vocab)
d_ff = 64 
dropout = 0.1 
length = sisdataset.SLF_max_length + sisdataset.SRnase_max_length



model = SisModle(N = N, d_model=d_model,d_ff=d_ff, dropout=dropout, seq_length=length) 


In [16]:
for x in train_dataloader:
    y_pred = model(x)   
    y_true = x["label"]
 
    print(y_pred)
    print(y_true)
    break 

/xtdisk/xueyb_group/xutingfeng/Anaconda/envs/torch/lib/python3.10/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[0.5080],
        [0.4833],
        [0.5013],
        [0.5058]], grad_fn=<SigmoidBackward0>)
tensor([0, 0, 0, 0])


In [20]:
c = copy.deepcopy

d_model = len(aa_vocab)
d_ff = 64
dropout = 0.1
N = 4
length = sisdataset.SLF_max_length + sisdataset.SRnase_max_length


onehot = OnehotLayer(d_model)
attn = trans.MultiHeadAttention(1, d_model)
ff = trans.PositionwiseFeedForward(d_model, d_ff,dropout)
encoderlayer = trans.EncoderLayer(d_model, c(attn), c(ff), dropout)
encoder_SLF = trans.Encoder(c(encoderlayer), N)
encoder_SRnase = trans.Encoder(c(encoderlayer), N)

fc1 = nn.Linear(d_model, 1)
fc2 = nn.Linear(length, 1)


for x in train_dataloader:
    x_SLF = onehot(x["SLF_Seq_token"]).float()
    x_SRnase = onehot(x["SRnase_Seq_token"]).float()
    y = x["label"]
    x_SLF = fc1(encoder_SLF(x_SLF, None))
    x_SRnase = fc1(encoder_SRnase(x_SRnase, None))
    
    o = torch.concat([x_SLF, x_SRnase], dim=1).squeeze(-1)
    
    last_mask = torch.concat([x["SLF_Seq_mask"], x["SRnase_Seq_mask"]], dim=1)
    o = fc2(o.masked_fill(last_mask, 1e-9))
    print(o)

    break 

torch.Size([4, 1])


tensor([[4.0277e-01, 2.3571e-01, 4.9383e-01,  ..., 1.0000e-09, 1.0000e-09,
         1.0000e-09],
        [8.9821e-01, 8.3182e-01, 1.1953e+00,  ..., 1.0000e-09, 1.0000e-09,
         1.0000e-09],
        [9.5022e-01, 1.0824e+00, 6.6873e-01,  ..., 1.0000e-09, 1.0000e-09,
         1.0000e-09],
        [1.0430e+00, 8.0308e-01, 6.4187e-01,  ..., 1.0000e-09, 1.0000e-09,
         1.0000e-09]], grad_fn=<MaskedFillBackward0>)

In [1]:
import sys
sys.path.append("/p300s/wangmx_group/xutingfeng/SIS/")

from sis.dataset import SISDataset
import torch 
import transformers 
from sis.model.sismodel import DoubleTransformerModel

sisdataset = SISDataset(root_dir="/p300s/wangmx_group/xutingfeng/SIS/sis/dataset/total_data.csv")

aa_vocab = sisdataset.aa_vocab
sis_datasetDict = sisdataset.dataset_dict

train_dataloader = torch.utils.data.DataLoader(sis_datasetDict["train"], batch_size=4)
test_dataloader = torch.utils.data.DataLoader(sis_datasetDict["test"], batch_size=4)

N = 4 
d_model = len(aa_vocab)
d_ff = 64 
dropout = 0.1 
length = sisdataset.SLF_max_length + sisdataset.SRnase_max_length

model = DoubleTransformerModel(N = N, d_model=d_model,d_ff=d_ff, dropout=dropout, seq_length=length) 


for x in train_dataloader:
    y_pred = model(x)   
    y_true = x["label"]
 
    print(y_pred)
    print(y_true)
    break 

Using custom data configuration default-52200760334198e7
Reusing dataset csv (/home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-bec2fd355b0cb7d7.arrow


Loading cached processed dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-0f1986b4010acd7d.arrow


Loading cached processed dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-fbd879c251d0f77b.arrow


Loading cached processed dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-d7b3eee632703ea9.arrow
Loading cached split indices for dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-fe11604753fbe9ea.arrow and /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-859c2defe8db73a3.arrow


tensor([[0.5153],
        [0.4990],
        [0.5033],
        [0.4975]], grad_fn=<SigmoidBackward0>)
tensor([0, 0, 0, 0])


/xtdisk/xueyb_group/xutingfeng/Anaconda/envs/torch/lib/python3.10/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [4]:
sis_datasetDict.set

DatasetDict({
    train: Dataset({
        features: ['SLF', 'SLF_Seq', 'SRnase', 'SRnase_Seq', 'label', 'SLF_Seq_token', 'SRnase_Seq_token', 'SLF_Seq_mask', 'SRnase_Seq_mask'],
        num_rows: 59
    })
    test: Dataset({
        features: ['SLF', 'SLF_Seq', 'SRnase', 'SRnase_Seq', 'label', 'SLF_Seq_token', 'SRnase_Seq_token', 'SLF_Seq_mask', 'SRnase_Seq_mask'],
        num_rows: 15
    })
})